In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize the simulator

In [3]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
D = 11
T = 11

OTHER_DATE = '2024-03-24'
DOUBLE_MSMT = True

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, D, T, DEVICE, _is_hex, OTHER_DATE, DOUBLE_MSMT)

12:27:08 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
12:27:08 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
12:27:08 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 17:49:53.348518+00:00.


In [4]:
simulator.noise_list

[0.012198849629634207,
 0.0004089493467995245,
 0.0016527862595933853,
 0.0020821405014924475,
 0.02578180156360313,
 0.019907289814579635,
 0]

# Needed decoding objects

In [5]:
# Get the KDEs
import numpy as np
import cpp_soft_info as csi
from Scratch import load_calibration_memory

bandwidths = [0.6]
num_points = 20
rel_error = 1

all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                    other_date=OTHER_DATE, post_process=True,
                                                    double_msmt=False)
kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)


all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=OTHER_DATE, post_process=True,
                                                                            double_msmt=True)
# kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 17:49:22.974787+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 17:49:53.348518+00:00.


In [6]:
from soft_info import inv_qubit_mapping

inverted_q_map = inv_qubit_mapping(simulator.qubit_mapping)

# Check if the simulator is using post selection
print(f"simulator qubit 20 shape: {simulator.all_memories[20]['mmr_0'].shape}")
print(f"Used memories for PS qubit 20 shape: {all_memories_PS[20]['mmr_0'].shape}")


simulator qubit 20 shape: (9411,)
Used memories for PS qubit 20 shape: (9411,)


# Get IQ Data

In [7]:
XBASIS = False
RESETS = False
LOGICAL = '0'

SHOTS = int(1e4)
INDIVIDUAL = False

IQ_data = simulator.generate_IQ(SHOTS, LOGICAL, XBASIS, RESETS, INDIVIDUAL)

In [8]:
simulator.noise_list

[0.012198849629634207,
 0.0004089493467995245,
 0.0016527862595933853,
 0.0020821405014924475,
 0.02578180156360313,
 0.019907289814579635,
 0]

# Decode

In [9]:
# Generate pSoft and countMat
HANDLE_OUTLIERS = True

pSoft, countMat = csi.iqConvertor(IQ_data, inverted_q_map, kde_dict, rel_error, -1, HANDLE_OUTLIERS)
# pSoft_PS, countMat_PS = csi.iqConvertor(IQ_data, inverted_q_map, kde_dict_PS, rel_error, -1, HANDLE_OUTLIERS)

In [10]:
p_soft, p_hard = 0, 0
for value in simulator.msmt_err_dict.values():
    p_soft += value['p_soft']
    p_hard += value['p_hard']
p_soft /= len(simulator.msmt_err_dict)
p_hard /= len(simulator.msmt_err_dict)
print(f"p_soft: {p_soft}, p_hard: {p_hard}")

p_soft: 0.005874511749023496, p_hard: 0.019907289814579635


In [11]:
# Initialize the stim circuits
from soft_info import RepetitionCodeStimCircuit

print(f"simulator noise_list: {simulator.noise_list}")

# Soft
noise_list_soft = simulator.noise_list.copy()
circ_soft = RepetitionCodeStimCircuit(D, T, XBASIS, RESETS, noise_list=noise_list_soft, no_fin_soft=True)
model_soft = circ_soft.circuits[LOGICAL].detector_error_model()


noise_list_hard_mean = simulator.noise_list.copy()
noise_list_hard_mean[-1]=np.mean(pSoft)
circ_hard_mean = RepetitionCodeStimCircuit(D, T, XBASIS, RESETS, noise_list=noise_list_hard_mean, no_fin_soft=False)
model_hard_mean = circ_hard_mean.circuits[LOGICAL].detector_error_model()

noise_list_hard = simulator.noise_list.copy()
noise_list_hard[-1]=p_soft
circ_hard = RepetitionCodeStimCircuit(D, T, XBASIS, RESETS, noise_list=noise_list_hard, no_fin_soft=False)
model_hard = circ_hard.circuits[LOGICAL].detector_error_model()



print(f"noise_list_soft: {noise_list_soft}")
print(f"noise_list_hard_mean: {noise_list_hard_mean}")
print(f"noise_list_hard: {noise_list_hard}")
print(f"simulator noise_list: {simulator.noise_list}")
print("[two, single, t1, t2, readout, p_hard, p_soft]")

simulator noise_list: [0.012198849629634207, 0.0004089493467995245, 0.0016527862595933853, 0.0020821405014924475, 0.02578180156360313, 0.019907289814579635, 0]
noise_list_soft: [0.012198849629634207, 0.0004089493467995245, 0.0016527862595933853, 0.0020821405014924475, 0.02578180156360313, 0.019907289814579635, 0]
noise_list_hard_mean: [0.012198849629634207, 0.0004089493467995245, 0.0016527862595933853, 0.0020821405014924475, 0.02578180156360313, 0.019907289814579635, 0.01635978004440409]
noise_list_hard: [0.012198849629634207, 0.0004089493467995245, 0.0016527862595933853, 0.0020821405014924475, 0.02578180156360313, 0.019907289814579635, 0.005874511749023496]
simulator noise_list: [0.012198849629634207, 0.0004089493467995245, 0.0016527862595933853, 0.0020821405014924475, 0.02578180156360313, 0.019907289814579635, 0]
[two, single, t1, t2, readout, p_hard, p_soft]


In [12]:
# Decode
res_s_K = csi.decodeConvertorAll(model_soft, countMat, pSoft, T, int(LOGICAL), RESETS, decode_hard = False)
res_h_K_mean = csi.decodeConvertorAll(model_hard_mean, countMat, pSoft, T, int(LOGICAL), RESETS, decode_hard = True)
res_h_K = csi.decodeConvertorAll(model_hard, countMat, pSoft, T, int(LOGICAL), RESETS, decode_hard = True)

In [13]:
print(f"D: {D}, T: {T}, SHOTS: {SHOTS}, LOGICAL: {LOGICAL}, XBASIS: {XBASIS}, RESETS: {RESETS}, INDIVIDUAL: {INDIVIDUAL}, DOUBLE_MSMTS: {DOUBLE_MSMT}\n")

print(f"soft KDE: {res_s_K.num_errors} out of {len(IQ_data)} shots, handleOutliers: {HANDLE_OUTLIERS}")
print(f"hard KDE mean: {res_h_K_mean.num_errors} out of {len(IQ_data)} shots, handleOutliers: {HANDLE_OUTLIERS}")
print(f"hard KDE: {res_h_K.num_errors} out of {len(IQ_data)} shots, handleOutliers: {HANDLE_OUTLIERS}")

print()

D: 11, T: 11, SHOTS: 10000, LOGICAL: 0, XBASIS: False, RESETS: False, INDIVIDUAL: False, DOUBLE_MSMTS: True

soft KDE: 243 out of 10000 shots, handleOutliers: True
hard KDE mean: 195 out of 10000 shots, handleOutliers: True
hard KDE: 290 out of 10000 shots, handleOutliers: True

